# Trabalho Prático Disciplina Aplicações de Aprendizado de Máquina para IoT AMI211A
## MBA Machine Learning - IGTI
## Adriano Arruda Ramos

In [ ]:
# Kernel usado no trabalho
# Importação de bibliotecas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, LSTM
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# dataset usado: https://www.kaggle.com/volpatto/temperature-timeseries-for-some-brazilian-cities

In [ ]:
# Leitura de dados de temperatura de Belem
df_belem = pd.read_csv("../input/temperature-timeseries-for-some-brazilian-cities/station_belem.csv")
df_belem.head()

In [ ]:
# Leitura de dados de temperatura de Curitiba
df_curitiba = pd.read_csv("../input/temperature-timeseries-for-some-brazilian-cities/station_curitiba.csv")
df_curitiba.head()

### Atividades

In [ ]:
# Tamanho do dataset Belem
df_belem.shape

In [ ]:
# Tamanho dataset Curitiba
df_curitiba.shape

In [ ]:
df_belem.tail()

In [ ]:
df_curitiba.tail()

In [ ]:
df_belem.info()

In [ ]:
df_belem.describe()

In [ ]:
df_curitiba.info()

In [ ]:
df_curitiba.describe()

#### Minimizando o impacto dos outliers (999.9) em todo o dataset para Belem e Curitiba.
##### Para minimizar o impacto dos valores outliers 999.9 em todo o dataset, irei criar um novo dataset com valores nulos no lugar dos valores 999.9 e em seguida preencher os nulos com valores médios.

In [ ]:
df_belem_sem_outliers = df_belem.replace(999.90,np.nan)
df_belem_sem_outliers = df_belem_sem_outliers.fillna(df_belem_sem_outliers.mean())
df_belem_sem_outliers.describe()

In [ ]:
df_curitiba_sem_outliers = df_curitiba.replace(999.90,np.nan)
df_curitiba_sem_outliers = df_curitiba_sem_outliers.fillna(df_curitiba_sem_outliers.mean())
df_curitiba_sem_outliers.describe()

In [ ]:
# Plotando o gráfico para o mês de Julho com comparação entre Belem e Curitiba, usando scatter plot
plt.figure(figsize=(10,10))
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(df_belem_sem_outliers.YEAR, df_belem_sem_outliers.JUL, color='r')
ax.scatter(df_curitiba_sem_outliers.YEAR, df_curitiba_sem_outliers.JUL, color='b')
ax.set_xlabel('Ano')
ax.set_ylabel('Temperatura (ºC)')
ax.legend(["Belém - Julho", "Curitiba - Julho"])
ax.set_title('scatter plot')
plt.show()

In [ ]:
df_belem_sem_outliers["JUL"].describe()

In [ ]:
df_curitiba_sem_outliers["JUL"].describe()

In [ ]:
#aplicando o teste estatístico ANOVA para comparar temperaturas de JUL entre as cidades
import scipy.stats as stats

In [ ]:
#stats.f_oneway(data_group1, data_group2, data_group3, data_groupN)
stats.f_oneway(df_belem_sem_outliers["JUL"], df_curitiba_sem_outliers["JUL"])

### Criando um modelo de previsão de séries temporais para a cidade de Curitiba em janeiro

In [ ]:
#coleta a serie temporal para o modelo de previsão
df_curitiba_sem_outliers_jan = df_curitiba_sem_outliers["JAN"].values
df_curitiba_sem_outliers_jan

In [ ]:
# normalização dos dados
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
serie_curitiba_jan = scaler.fit_transform(df_curitiba_sem_outliers_jan.reshape(-1, 1))
serie_curitiba_jan

In [ ]:
#define a quantidade de dados para treinamento e teste
train=serie_curitiba_jan[:int(len(serie_curitiba_jan)*0.7)]
test=serie_curitiba_jan[int(len(serie_curitiba_jan)*0.7):]

In [ ]:
train

In [ ]:
test

In [ ]:
# comprimento do teste e treinamento
print(len(train), len(test))

In [ ]:
#define o intervalo a ser utilizado para entrada da rede neural LSTM
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX,trainY

In [ ]:
#colocando os dados no formato para entrada na rede LSTM em deep learning [amostras, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], look_back, 1))
testX = np.reshape(testX, (testX.shape[0], look_back, 1))

#### Definindo o modelo LSTM

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(look_back,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
#mostra o modelo
model.summary()

In [ ]:
#Treina o modelo
model.fit(trainX, trainY, epochs=5, batch_size=1, verbose=2)

In [ ]:
# realizando as previsões
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# invertendo as previsões
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
import math
from sklearn.metrics import mean_squared_error

In [ ]:
# Calculando os erros de previsão
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Pontuação para o treinamento: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Pontuação para o teste: %.2f RMSE' % (testScore))

In [ ]:
# realizando o shift dos valores para construir o plot dos valores
trainPredictPlot = np.empty_like(serie_curitiba_jan)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

In [ ]:
# realizando m shift dos dados de previsão para o treinamento
testPredictPlot = np.empty_like(serie_curitiba_jan)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(serie_curitiba_jan)-1, :] = testPredict

In [ ]:
# realizando o plot dos dados para treinamento e teste
plt.plot(scaler.inverse_transform(serie_curitiba_jan))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.legend(["Real","Previsao Treinamento","Previsão Teste"])
plt.show()